In [1]:
import pandas as pd
import numpy as np

USER_HISTORY_PATH = "../data/raw/user_history.csv"
NOTIF_PATH = "../data/raw/notification_allowed.csv"

user_history = pd.read_csv(USER_HISTORY_PATH)
notif = pd.read_csv(NOTIF_PATH)

In [33]:
print("notifications shape:", notif.shape)
print("user history shape:", user_history.shape)

print("\nuser history columns:", len(user_history.columns))
print("notifications columns:", len(notif.columns))

print("\nunique users in user history:", user_history["login_id"].nunique())
print("unique users in notifications:", notif["login_id"].nunique())

print("\nuser history date_id range:", user_history["date_id"].min(), "->", user_history["date_id"].max())
print("notificatons date_id range:", notif["date_id"].min(), "->", notif["date_id"].max())
print("registration date range:", user_history["registration_date_id"].min(), "->", user_history["registration_date_id"].max())

print("\nuser history nan values:", user_history.isna().any().any())
print("notifications nan values:", user_history.isna().any().any())

counter = notif.groupby("login_id").size()
print("\nnotification records per user:")
print(counter.value_counts().head(10))

display(user_history.head(3))
display(notif.head(3))

notifications shape: (3859, 4)
user history shape: (122459, 79)

user history columns: 79
notifications columns: 4

unique users in user history: 4979
unique users in notifications: 3859

user history date_id range: 0 -> 34
notificatons date_id range: 0 -> 22
registration date range: 0 -> 13

user history nan values: False
notifications nan values: False

notification records per user:
1    3859
Name: count, dtype: int64


,login_id,registration_date_id,registration_channel,registration_country,payer,dau,sessions_count,playtime,last_login_day,days_active_last_7_days,...,equipment_bonus,campaign_battles,campaign_battles_total,event_battles,event_battles_total,event_tokens,group_points,lives_spent,milestone_points,date_id
0,1,0,Organic,Canada,False,1,3,1351,0,1,...,0.0,5,5,0,0,15,0,0,0,0
1,2,0,Organic,Brazil,False,1,1,1046,0,1,...,0.0,2,2,0,0,15,0,0,0,0
2,3,0,Organic,Brazil,False,1,7,5854,0,1,...,0.0,8,8,0,0,40,0,0,0,0


,login_id,time,allowed_notifications,date_id
0,352,999,True,0
1,184,2780,True,0
2,162,2928,False,0


In [21]:
user_history[user_history['date_id'] == 1]

,login_id,registration_date_id,registration_channel,registration_country,payer,dau,sessions_count,playtime,last_login_day,days_active_last_7_days,...,equipment_bonus,campaign_battles,campaign_battles_total,event_battles,event_battles_total,event_tokens,group_points,lives_spent,milestone_points,date_id
381,1,0,Organic,Canada,False,0,0,0,0,1,...,0.0,0,5,0,0,15,0,0,0,1
382,2,0,Organic,Brazil,False,0,0,0,0,1,...,0.0,0,2,0,0,15,0,0,0,1
383,3,0,Organic,Brazil,False,1,3,1280,1,2,...,0.0,0,8,0,0,55,0,0,0,1
384,4,0,Paid,France,False,0,0,0,0,1,...,0.0,0,1,0,0,40,0,0,0,1
385,5,0,Paid,United Kingdom,False,0,0,0,0,1,...,0.0,0,2,0,0,5,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1165,785,1,Organic,Brazil,False,1,16,3631,1,1,...,0.0,11,11,0,0,40,0,0,0,1
1166,786,1,Organic,United States,False,1,1,1053,1,1,...,0.0,3,3,0,0,15,0,0,0,1
1167,787,1,Paid,Germany,False,1,6,5648,1,1,...,0.0,7,7,0,0,40,0,0,0,1
1168,788,1,Paid,United States,False,1,3,2959,1,1,...,0.0,7,7,0,0,15,0,0,0,1


```user_history date_id```
- dnevni snapshot-ovi
- kako se igrac ponasao tog dana
- Zasto ide do 34, a test traje 14 dana?
    - test traje od 0 do 13, ali ako se user registruje 13og dana
    - on mora da gleda sledecih 14 dana
    - zato bi trebalo da ide od 27, ali zbog pravila (21 dan)
        - ide do 34
 
```registration date id```
- dan kada se user prvi put pojavio u igri

- Ukupno usera na testu ima: 4.979
- Popup odgovor imamo za 3.859
    - 1.120 igraca nikad nije ni doslo da popupa!

## Group assignment

In [25]:
users = user_history.copy()
users['group'] = np.where(users["login_id"] % 2 == 0, "group1_even", "group2_odd")

user_base = (
    users
    .sort_values(["login_id", "date_id"])
    .groupby("login_id", as_index=False)
    .first()
)

In [29]:
print('user_base shape:', user_base.shape)
display(user_base[[
    "login_id",
    "group",
    "registration_date_id",
    "registration_channel",
    "registration_country",
    "payer",
    "date_id"
]].head(4))

user_base shape: (4979, 80)


,login_id,group,registration_date_id,registration_channel,registration_country,payer,date_id
0,1,group2_odd,0,Organic,Canada,False,0
1,2,group1_even,0,Organic,Brazil,False,0
2,3,group2_odd,0,Organic,Brazil,False,0
3,4,group1_even,0,Paid,France,False,0


## Merge

In [34]:
user_base = user_base.merge(
    notif[["login_id", "allowed_notifications", "date_id"]],
    on="login_id",
    how="left",
    suffixes=("","_notif")
)

In [40]:
user_base.isna().any()

login_id                 False
registration_date_id     False
registration_channel     False
registration_country     False
payer                    False
                         ...  
milestone_points         False
date_id                  False
group                    False
allowed_notifications     True
date_id_notif             True
Length: 82, dtype: bool

## Exposed notifications

In [43]:
user_base["notif_exposed"] = user_base["allowed_notifications"].notna()

In [46]:
user_base["notif_exposed"].sum()

np.int64(3859)

## Allowed notifications

In [47]:
user_base["allowed"] = (
    user_base["allowed_notifications"]
    .astype('boolean')
    .fillna(False)
    .astype(int)
)

In [50]:
print("After merge:")
print(user_base.shape)

print("\nExposure rate:")
print(user_base["notif_exposed"].value_counts(), '\n')
print(user_base["notif_exposed"].value_counts(normalize=True), '\n')

print(user_base["allowed"].value_counts(), '\n')
print(user_base["allowed"].value_counts(normalize=True))


display(
    user_base[[
        "login_id",
        "group",
        "notif_exposed",
        "allowed",
        "date_id_notif"
    ]]
)

After merge:
(4979, 84)

Exposure rate:
notif_exposed
True     3859
False    1120
Name: count, dtype: int64 

notif_exposed
True     0.775055
False    0.224945
Name: proportion, dtype: float64 

allowed
0    2684
1    2295
Name: count, dtype: int64 

allowed
0    0.539064
1    0.460936
Name: proportion, dtype: float64


,login_id,group,notif_exposed,allowed,date_id_notif
0,1,group2_odd,False,0,NaN
1,2,group1_even,True,1,0.0
2,3,group2_odd,True,0,0.0
3,4,group1_even,True,0,0.0
4,5,group2_odd,False,0,NaN
...,...,...,...,...,...
4974,4975,group2_odd,True,1,13.0
4975,4976,group1_even,True,0,13.0
4976,4977,group2_odd,False,0,NaN
4977,4978,group1_even,True,1,13.0


Skoro cetvrtina igraca nikad nije dosla do trenutka kad se popup prikazuje\
Zasto?
- odustali su pre tutorijala, verovatno narocito u group2
- sad cemo to da proverimo


In [51]:
exposure_by_group = (
    user_base
    .groupby("group")["notif_exposed"]
    .agg(
        total_users = "count",
        exposed_users = "sum",
        exposure_rate = "mean"
    )
)

display(exposure_by_group)

,total_users,exposed_users,exposure_rate
group,,,
group1_even,2489,2487,0.999196
group2_odd,2490,1372,0.551004


Potvrdjujemo gore navedeno:
- grupa kojoj se odmah pokazuje notif: 99.9% vidi popup
- grupa kojoj se pokazuje notif nakon tutorijala: 55.1%

## SRM - Sample Ratio MisMatch

In [52]:
group_counts = user_base['group'].value_counts()
display(group_counts)

print('\nGroup proportions:')
print(group_counts / group_counts.sum())

group
group2_odd     2490
group1_even    2489
Name: count, dtype: int64


Group proportions:
group
group2_odd     0.5001
group1_even    0.4999
Name: count, dtype: float64


### p-value

In [56]:
from scipy.stats import chisquare

observed = group_counts.values
expected = [group_counts.sum() / 2, group_counts.sum() / 2]

chi2, pval = chisquare(observed, expected)

print("SRM chi-square test:")
print("chi2 = ", chi2)
print("p-value = ", pval)

SRM chi-square test:
chi2 =  0.0002008435428800964
p-value =  0.9886928159591263


- Nema dokaza da je split pokvaren, tj mozemo verovati rezultatima eksperimenta

```chi-square test```
- da li se rezultat znacajno razlikuje od onoga sto ockeujemo

```ITT```
- racunamo metriku nad svim userima koji su dodeljeni grupi
    - cak i ako nisu videi popup
- Zasto?
    - meri realan efekat u produkciji

In [57]:
hit_rate_itt = (
    user_base
    .groupby('group')['allowed']
    .agg(
        total_users = "count",
        allowed_users = "sum",
        hit_rate = "mean"
    )
)

hit_rate_itt["hit_rate_percent"] = hit_rate_itt["hit_rate"]*100

display(hit_rate_itt)

,total_users,allowed_users,hit_rate,hit_rate_percent
group,,,,
group1_even,2489,1434,0.576135,57.613499
group2_odd,2490,861,0.345783,34.578313


Razlika je +23 procenta u korist **ranog** popupa

## Statisticka znacajnost razlike

In [63]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

success = hit_rate_itt["allowed_users"].values
nobs = hit_rate_itt["total_users"].values

zstat, pval = proportions_ztest(success, nobs)

print("z-stat", zstat)
print("p-value", pval)

ci_group1 = proportion_confint(success[0], nobs[0], alpha=0.05, method="normal")
ci_group2 = proportion_confint(success[1], nobs[1], alpha=0.05, method="normal")

print("\n95% CI group1:", ci_group1)
print("\n95% CI group2:", ci_group2)

diff = hit_rate_itt.loc["group1_even", "hit_rate"] - hit_rate_itt.loc["group2_odd", "hit_rate"]
print("\nHit rate difference (group1 - group2):", diff)

z-stat 16.303930312986633
p-value 9.254491215024052e-60

95% CI group1: (0.5567211492554596, 0.5955488386915071)

95% CI group2: (0.32710165644643946, 0.36446460861380153)

Hit rate difference (group1 - group2): 0.23035186144336284


### Statisticka znacajnost

**Highly statistically significant (p < 0.001)**

---

### Zakljucak

- Prikazivanje popup poruke za dozvolu push notifikacija odmah nakon instalacije znacajno povecava stopu prihvatanja za oko **23 procentna poena** u odnosu na prikazivanje posle tutorijala
- Razlika je **veoma statisticki znacajna (p < 0.001)**

---


Na prethodnom testu analizirani su svi korisnici i rezultat je pokazao **prednost grupe 1**. \
Medjutim, kada posmatramo samo korisnike koji su zaista bili izlozeni popup poruci, obrazac se blago menja i dobijamo nesto drugaciji ishod.

In [64]:
exposed = user_base[user_base["notif_exposed"] == True]

hit_rate_exposed = (
    exposed
    .groupby("group")["allowed"]
    .agg(
        exposed_users = "count",
        allowed_users = "sum",
        hit_rate = "mean"
    )
)

hit_rate_exposed["hit_ratehit_percent"] = hit_rate_exposed["hit_rate"]*100

display(hit_rate_exposed)

,exposed_users,allowed_users,hit_rate,hit_ratehit_percent
group,,,,
group1_even,2487,1434,0.576598,57.659831
group2_odd,1372,861,0.627551,62.755102


Kada user stigne do **pop-up** posle tutorijala, veca je sansa da klikne Allow
- Zasto?
    - vec je dosta vremena ulozio u gledanje tutorijala
    - svidela mu se igra cim je toliko ostao

---
***Iako popup prikazan nakon tutorijala ima bolji rezultat medju korisnicima koji su mu bili izlozeni (oko 5 procentnih poena veci opt-in), prikazivanje odmah nakon instalacije dopire do skoro duplo vise igraca, sto na kraju dovodi do znatno vece ukupne stope prihvatanja.***

---